In [1]:
import numpy as np

def quatmult(q1, q2):
    """
    Multiply two quaternions.
    
    Parameters:
    q1, q2 : numpy arrays of shape (4,)
        Quaternions in form [w, x, y, z] where w is the scalar part.
        
    Returns:
    numpy array of shape (4,)
        The quaternion product q1 * q2.
    """
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    z = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2
    
    return np.array([w, x, y, z])

def quaternion_conjugate(q):
    """
    Compute the conjugate of a quaternion.
    
    Parameters:
    q : numpy array of shape (4,)
        Quaternion in form [w, x, y, z].
        
    Returns:
    numpy array of shape (4,)
        The conjugate quaternion [w, -x, -y, -z].
    """
    return np.array([q[0], -q[1], -q[2], -q[3]])

def rotate_quaternion_by_quaternion(q, r):
    """
    Rotate a quaternion q by another quaternion r.
    
    The rotation is performed as: q' = r * q * r^(-1)
    
    Parameters:
    q : numpy array of shape (4,)
        The quaternion to rotate, in form [w, x, y, z].
    r : numpy array of shape (4,)
        The rotation quaternion, in form [w, x, y, z].
        Should be a unit quaternion (normalized).
        
    Returns:
    numpy array of shape (4,)
        The rotated quaternion.
    """
    # Ensure r is normalized
    r = r / np.linalg.norm(r)
    
    # Compute r^(-1), which is the conjugate for unit quaternions
    r_inv = quaternion_conjugate(r)
    
    # Perform rotation: q' = r * q * r^(-1)
    rotated_q = quatmult(quatmult(r, q), r_inv)
    
    return rotated_q

def matrix_to_quaternion(R):
    """
    Convert a 3x3 rotation matrix to a quaternion.
    
    Parameters:
    R : numpy array of shape (3, 3)
        Rotation matrix.
        
    Returns:
    numpy array of shape (4,)
        The quaternion representation in form [w, x, y, z].
    """
    trace = np.trace(R)
    
    if trace > 0:
        S = np.sqrt(trace + 1.0) * 2
        w = 0.25 * S
        x = (R[2, 1] - R[1, 2]) / S
        y = (R[0, 2] - R[2, 0]) / S
        z = (R[1, 0] - R[0, 1]) / S
    elif R[0, 0] > R[1, 1] and R[0, 0] > R[2, 2]:
        S = np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2]) * 2
        w = (R[2, 1] - R[1, 2]) / S
        x = 0.25 * S
        y = (R[0, 1] + R[1, 0]) / S
        z = (R[0, 2] + R[2, 0]) / S
    elif R[1, 1] > R[2, 2]:
        S = np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2]) * 2
        w = (R[0, 2] - R[2, 0]) / S
        x = (R[0, 1] + R[1, 0]) / S
        y = 0.25 * S
        z = (R[1, 2] + R[2, 1]) / S
    else:
        S = np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1]) * 2
        w = (R[1, 0] - R[0, 1]) / S
        x = (R[0, 2] + R[2, 0]) / S
        y = (R[1, 2] + R[2, 1]) / S
        z = 0.25 * S
    
    return np.array([w, x, y, z])

def rotate_quaternion_by_matrix(q, R):
    """
    Rotate a quaternion by a 3x3 rotation matrix.
    
    Parameters:
    q : numpy array of shape (4,)
        The quaternion to rotate, in form [w, x, y, z].
    R : numpy array of shape (3, 3)
        The rotation matrix.
        
    Returns:
    numpy array of shape (4,)
        The rotated quaternion.
    """
    # Convert rotation matrix to quaternion
    r = matrix_to_quaternion(R)
    
    # Use the quaternion rotation function
    return rotate_quaternion_by_quaternion(q, r)


In [8]:

# Define a quaternion representing a point or orientation
q = np.array([1.0, 0.0, 0.0, 0.0])  # Identity quaternion

# Define a rotation quaternion (45 degrees around z-axis)
angle = np.pi/3
axis = np.array([0, 1, 1])
axis = axis / np.linalg.norm(axis)
r = np.array([np.cos(angle/2), *np.sin(angle/2) * axis])

# Rotate the quaternion by another quaternion
rotated_q = rotate_quaternion_by_quaternion(q, r)
print("Rotated by quaternion:", rotated_q)

# Create a rotation matrix (45 degrees around z-axis)
c = np.cos(angle)
s = np.sin(angle)
R = np.array([
    [c, -s, 0],
    [s, c, 0],
    [0, 0, 1]
])

# Rotate the quaternion by the rotation matrix
rotated_q_by_matrix = rotate_quaternion_by_matrix(q, R)
print("Rotated by matrix:", rotated_q_by_matrix)

Rotated by quaternion: [1. 0. 0. 0.]
Rotated by matrix: [1. 0. 0. 0.]


In [12]:
q = np.array([1.0, 0.0, 0.0, 0.0])  # Identity quaternion
q2 = np.array([0.7071, 0, 0, -0.7071])
rotate_quaternion_by_quaternion(q2, q)

array([ 0.7071,  0.    ,  0.    , -0.7071])

In [27]:
import numpy as np

def quat_multiply(q1, q2):
    """Multiplies two quaternions."""
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return np.array([
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ])

def quat_conjugate(q):
    """Returns the conjugate of a quaternion."""
    w, x, y, z = q
    return np.array([w, -x, -y, -z])

def rotate_quat_by_quat(q, r):
    """Rotates quaternion q by quaternion r (r * q * r^-1)."""
    r_inv = quat_conjugate(r)
    return quat_multiply(quat_multiply(r, q), r_inv)

# Example:
q = np.array([1, 0, 0, 0])  # A quaternion
r = np.array([0.7071, 0, 0, -0.7071])  # A quaternion that represents a rotation
q = rotate_quat_by_quat(q, r)
print("Rotated Quaternion:", q)


Rotated Quaternion: [0.99998082 0.         0.         0.        ]


In [15]:
import numpy as np

def quatmult(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    w = w1*w2 - x1*x2 - y1*y2 - z1*z2
    x = w1*x2 + x1*w2 + y1*z2 - z1*y2
    y = w1*y2 - x1*z2 + y1*w2 + z1*x2
    z = w1*z2 + x1*y2 - y1*x2 + z1*w2
    return np.array([w, x, y, z])

def quaternion_conjugate(q):
    return np.array([q[0], -q[1], -q[2], -q[3]])

# Known values
q_base = np.array([1.0, 0.0, 0.0, 0.0])  # Example: base frame aligned with global frame
q_IMU = np.array([0.7071, 0.0, 0.0, -0.7071])  # Example: IMU reports 90° about Z
q_offset = np.array([0.7071, 0.0, 0.0, 0.7071])  # 90° rotation about Z

# Compute inverses
q_base_inv = quaternion_conjugate(q_base)  # Inverse of base quaternion
q_offset_inv = quaternion_conjugate(q_offset)  # Inverse of offset quaternion

# Compute desired IMU quaternion
q_IMU_relative_to_base = quatmult(q_offset_inv, quatmult(q_base_inv, q_IMU))

print(f"Base quaternion: {q_base}")
print(f"Reported IMU quaternion: {q_IMU}")
print(f"Desired IMU quaternion (relative to base): {q_IMU_relative_to_base}")

Base quaternion: [1. 0. 0. 0.]
Reported IMU quaternion: [ 0.7071  0.      0.     -0.7071]
Desired IMU quaternion (relative to base): [ 0.          0.          0.         -0.99998082]
